In [2]:
import numpy as np
import pandas as pd
import pickle
from tqdm import tqdm
import gc, os
import logging
import time
from datetime import datetime
import lightgbm as lgb
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')

- 思路の整理概述：
> 1. 文章的自身特征: category_id表示这文章的类型， created_at_ts表示文章建立的时间， 这个关系着文章的时效性， words_count是文章的字数， 一般字数太长我们不太喜欢点击, 也不排除有人就喜欢读长文；
> 2. 文章的内容embedding特征: 这个召回的时候用过， 这里可以选择使用， 也可以选择不用， 也可以尝试其他类型的embedding特征， 比如W2V等；
> 3. 用户的设备特征信息
- 上面这些直接可以用的特征， 待做完特征工程之后， 直接就可以根据article_id或者是user_id把这些特征加入进去。 但是我们需要先基于召回的结果，构造一些特征，然后制作标签，形成一个监督学习的数据集。
- 构造监督数据集的思路， 根据召回结果， 我们会得到一个{user_id: [可能点击的文章列表]}形式的字典。 那么我们就可以对于每个用户， 每篇可能点击的文章构造一个监督测试集， 比如对于用户user1， 假设得到的他的召回列表{user1: [item1, item2, item3]}， 我们就可以得到三行数据(user1, item1), (user1, item2), (user1, item3)的形式， 这就是监督测试集时候的前两列特征。

构造特征的思路如下：
构造特征的思路是这样， 我们知道每个用户的点击文章是与其历史点击的文章信息是有很大关联的， 比如同一个主题， 相似等等。 
所以特征构造这块很重要的一系列特征是：——————————————————要结合用户的历史点击文章信息。
- 目前的召回结果：我们已经得到的就是用户: [item1, item2, item3]列表，以及相应得分；
- 而目的是要预测最后一次点击的文章，比较自然的一个思路就是和其最后几次点击的文章产生关系，这样既考虑了其历史点击文章信息，又得离最后一次点击较近；——源于用户兴趣标签可能距离时间相近的更有高权重：
> 1. 候选item与最后几次点击的相似性特征(embedding内积） — 这个直接关联用户历史行为
> 2. 候选item与最后几次点击的相似性特征的统计特征 — 统计特征可以减少一些波动和异常
> 3. 候选item与最后几次点击文章的字数差的特征 — 可以通过字数看用户偏好
> 4. 候选item与最后几次点击的文章建立的时间差特征 — 时间差特征可以看出该用户对于文章的实时性的偏好


- 最终确定的整体思路：
> 1. 我们首先获得用户的最后一次点击操作和用户的历史点击， 这个基于我们的日志数据集做
> 2. 基于用户的历史行为制作特征， 这个会用到用户的历史点击表， 最后的召回列表， 文章的信息表和embedding向量
> 3. 制作标签， 形成最后的监督学习数据集

### 目录：
- 序章：
> 压缩内存的函数&定义数据存储的路径：
- C1. 数据前期处理，准备：
> 1. S1.获取全数据集；
> 2. S2.定义验证集的采样流程；
> 3. S3.获取最后和历史点击；
> 4. S4.真正的训练、测试、验证集——调用S2中的验证集采样标准；
> 5. S5.读取召回列表——这里实际有效的只有ItemCF召回；
> 6. S6.读取信息embedding；
> 7. S7.读取文章信息；
> 8. S8.处理得到的数据；
> 9. S9.读取之前计算好的召回列表；
- C2. 特征工程环节之一——负采样处理——关键步骤；
> 1. P1.召回数据打Label；——转换成监督学习的前提；get_rank_label_df()打标签；
> 2. P2.定义负采样函数——neg_sample_recall_data()；
> 3. P3.通过召回列表，点击历史，验证历史，测试历史，末次点击训练；get_user_recall_item_label_df()：：
> 4. P4.最终的召回做字典存储；
- C3. 特征工程环节之二——用户历史行为，结合历史点击过的文章和召回的文章创造一些相似度上+词数+时间上的特征；——关键步骤；
- 反应一种用户触发相应的点击行为动作上的相似性；
- 最后的形式：user_id、click_article_id、sim0、time_diff0、word_diff0、sim_max、sim_min、sim_sum、sim_mean、sim_median、score、rank、label
> 1. P1.读取文章向量和文章基本属性信息：article_info_df = get_article_info_df()，articles_emb = get_embedding(save_path, all_click_df)
> 2. P2.基于点击的data做历史相关的特征；基于召回和最后一次点击的文章物品计算相似度，时间差等信息；
> 3. P3.获取训练验证及测试数据中召回列文章相关特征；
> 4. 做好的历史行为相关特征，进行存储；
形式为：user_id-click_article_id-sim0-time_diff0-word_diff0-sim_max-sim_min-sim_sum-sim_mean-sim_median-score-rank-label
>> 1. trn_user_item_feats_df = pd.read_csv(save_dir + 'trn_user_item_feats_df.csv')
>> 2. tst_user_item_feats_df = pd.read_csv(save_dir + 'tst_user_item_feats_df.csv')
- C4. 特征工程环节之三——用户/文章活跃热门特征构造处理；
> 1. P1.用户活跃度参数：——基于用户点击文章的篇数，平均间隔时间；
> 2. P2.文章热门度参数：——基于文章被点次数，被点时间间隔；
- C5. 特征工程环节之四——用户习惯偏好层面的数据挖掘——可以理解为用户画像的处理；
- 从all_click_df就是全部点击日志中，可以看到点击环境，时间，设备，地区，类型等信息；这个是不是可以视为所谓的用户画像？
- 用户的设备习惯， 这里取最常用的设备，最多的是哪个——取众数；
- 用户的时间习惯： 用户习惯啥时候看新闻和物品；
- 用户的爱好特征， 文章的类别、主题信息分类；
- 用户文章的词数， 用户的爱好文章的字数习惯，偏爱短文章长文章；
> 1. P1.用户设备习惯特征挖掘：拿出最多的！ 
> 2. P2.用户时间习惯特征挖掘：最习惯在几点点击浏览，最喜欢啥时候的！
> 3. P3.用户主题类别偏好特征挖掘：点击过啥主题类别的，并且召回的是否在点击的里面，最后做一个布尔值字段,is_cat_hub
> 4. P4.用户的文章词数偏好特征挖掘：用户点击过的文章词语数平均值
> 5. P5.用户层面的所有信息汇总；保存到指定路径：user_feature_all_info.to_csv(save_dir + 'user_feature_info.csv', index=False)  
- 最终字段解释：
<!-- 'user_id',           用户id；
'click_article_id',     点击文章id；
'sim0',             召回的文章物品与最后一次点击的文章之间的相似度；
'time_diff0',         召回的文章物品与最后一次点击的文章之间创建时间的差值；
'word_diff0',         召回的文章物品与最后一次点击的文章之间文章词数的差值；
'sim_max',           用户层面上召回的所有文章物品，与最后一次点击的文章之间的相似度最大值；
'sim_min',           用户层面上召回的所有文章物品，与最后一次点击的文章之间的相似度最小值；
'sim_sum',           用户层面上召回的所有文章物品，与最后一次点击的文章之间的相似度的Σ值；
'sim_mean',          用户层面上召回的所有文章物品，与最后一次点击的文章之间的相似度平均值；
'sim_median',         用户层面上召回的所有文章物品，与最后一次点击的文章之间的相似度中位值；
'score',            召回的文章物品分数；
'rank',             召回的文章物品与最后一次点击的文章之间的排序；
'click_size',         用户层面：点击次数取到数
'time_diff_mean',      用户层面：点击时间间隔平均值
'active_level_parameter', 用户活跃度参数，越大越活跃
'click_environment',    该用户最频繁的点击环境
'click_deviceGroup',    该用户最频繁的点击设备组
'click_os',          该用户最频繁的点击操作系统
'click_country',       该用户最频繁的点击城市
'click_region',       该用户最频繁的点击地区
'click_referrer_type',   该用户最频繁的点击来源类型
'click_weekday',       该用户最频繁的点击星期几
'click_hour',         该用户最频繁的点击几点
'article_crt_weekday',   该用户点击的文章创建最频繁周几
'article_crt_hour',     该用户点击的文章创建最频繁几点
'words_hbo',          该用户点击的文章词数      
'category_id',        该用户点击的文章类别
'words_count',        文章的词数
'is_cat_hab',         即召回文章的主题类别是否在用户的点击过的主题类别里面：即is_cat_hub字段；
'label'                是否是用户最后一次点击；——本项目中的特殊性，因为要求是最后一次点击所以要用最后一次click_trn_hist找到最后一次拼接标签；
——如果是正常情况只需要召回的文章物品中看点没点就行；
    -->

# 序章：定义内存节省函数

In [13]:
# 节约内存的一个标配函数，一种数据压缩技术
def reduce_mem(df):
    starttime = time.time()
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if pd.isnull(c_min) or pd.isnull(c_max):
                continue
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    print('-- Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction),time spend:{:2.2f} min'.format(end_mem,
                                                                                                           100*(start_mem-end_mem)/start_mem,
                                                                                                           (time.time()-starttime)/60))
    return df

# 序章：定义文件/结果路径

In [18]:
data_path = './data_path/'              # 原始数据存放位置
save_path = './recall_save_path/'       # 保存一些计算结果的文件，比如物品、用户相似度
save_dir = './feature_project_path/'    # 特征工程结果的存放位置

# C1.数据前期处理，准备
- 在训练集中抽取部分用户的所有信息来作为验证集。

### S1.获取全数据集

In [16]:
# 之前读取函数原封不动——先读取数据生成all_click_df训练集
def get_all_click_df(data_path = data_path, offline=True):
    if offline:
        all_click = pd.read_csv(data_path + 'train_click_log.csv')
    else:
        trn_click = pd.read_csv(data_path + 'train_click_log.csv')
        tst_click = pd.read_csv(data_path + 'testA_click_log.csv')

        all_click = trn_click.append(tst_click)    
    all_click = all_click.drop_duplicates((['user_id', 'click_article_id', 'click_timestamp']))
    return all_click
all_click_df = get_all_click_df()

### S2.定义验证集的采样流程；

In [17]:
# all_click_df指的是训练集
# sample_user_nums 采样作为验证集的用户数量
def trn_val_split(all_click_df, sample_user_nums):
    all_click = all_click_df
    all_user_ids = all_click.user_id.unique()
    # 随机抽取一部分用户作为验证集的样本；replace=True表示可以重复抽样，反之不可以
    sample_user_ids = np.random.choice(all_user_ids, size=sample_user_nums, replace=False) 
    # 随机采样的作为验证集样本
    click_val = all_click[all_click['user_id'].isin(sample_user_ids)]
    # 随机采样的以外数据作为训练集样本
    click_trn = all_click[~all_click['user_id'].isin(sample_user_ids)]
    # 将验证集点击顺序按时间排序
    click_val = click_val.sort_values(['user_id', 'click_timestamp'])
    # 将验证集中的最后一次点击给抽取出来作为答案结果集
    val_ans = click_val.groupby('user_id').tail(1)
    ############## 下面一部很关键！
    # 去除val_ans中某些用户只有一个点击数据的情况，如果该用户只有一个点击数据，又被分到答案结果集中
    # 那么训练集中就没有这个用户的点击数据，出现用户冷启动问题，给自己模型验证带来麻烦
    click_val = click_val.groupby('user_id').apply(lambda x: x[:-1]).reset_index(drop=True)
    val_ans = val_ans[val_ans.user_id.isin(click_val.user_id.unique())] # 保证答答案结果集中出现的用户在验证集中存在
    click_val = click_val[click_val.user_id.isin(val_ans.user_id.unique())]
    return click_trn, click_val, val_ans

### S3.最后和历史点击！

In [25]:
# 获取当前数据的历史点击和最后一次点击
def get_hist_and_last_click(all_click):
    all_click = all_click.sort_values(by=['user_id', 'click_timestamp'])
    click_last_df = all_click.groupby('user_id').tail(1)
    # 如果用户只有一个点击，hist为空了，会导致训练的时候这个用户不可见，此时默认泄露一下
    def hist_func(user_df):
        if len(user_df) == 1:
            return user_df
        else:
            return user_df[:-1]
    click_hist_df = all_click.groupby('user_id').apply(hist_func).reset_index(drop=True)
    return click_hist_df, click_last_df

### S4.真正的训练、测试、验证集——调用S2中的验证集采样标准

In [26]:
# 读取训练、验证及测试集
def get_trn_val_tst_data(data_path, offline=True):
    if offline:
        click_trn_data = pd.read_csv(data_path+'train_click_log.csv')  # 训练集用户点击日志
        click_trn_data = reduce_mem(click_trn_data) # 减少内存
        click_trn, click_val, val_ans = trn_val_split(click_trn_data , sample_user_nums) # 调用上面编写的采样标准
    else:
        click_trn = pd.read_csv(data_path+'train_click_log.csv')
        click_trn = reduce_mem(click_trn)
        click_val = None                                               # 线上比赛就无需这样弄了
        val_ans = None                                                 
    click_tst = pd.read_csv(data_path+'testA_click_log.csv')           # 最终测试集
    return click_trn, click_val, click_tst, val_ans                    # 返回点击训练集、点击验证集、点击测试集、答案结果集

### S5.读取召回列表——这里实际有效的只有ItemCF召回

In [27]:
# 返回多路召回列表或者单路召回
def get_recall_list(save_path, single_recall_model=None, multi_recall=False):
    if multi_recall:
        return pickle.load(open(save_path + 'final_recall_items_dict.pkl', 'rb'))
    if single_recall_model == 'i2i_itemcf':
        return pickle.load(open(save_path + 'itemcf_recall_dict.pkl', 'rb'))
    elif single_recall_model == 'i2i_emb_itemcf':
        return pickle.load(open(save_path + 'itemcf_emb_dict.pkl', 'rb'))

### S6.读取信息embedding

In [28]:
# 可以通过字典查询对应的item的Embedding
def get_embedding(save_path, all_click_df):
    if os.path.exists(save_path + 'item_content_emb.pkl'): ## 读取前文获得的文章向量信息
        item_content_emb_dict = pickle.load(open(save_path + 'item_content_emb.pkl', 'rb'))
    else:
        print('item_content_emb.pkl 文件不存在...')
#     # w2v Embedding是需要提前训练好的
#     if os.path.exists(save_path + 'emb_i2i_sim.pkl'):      ## 读取处理好的物品itemEmbedding信息
#         item_w2v_emb_dict = pickle.load(open(save_path + 'emb_i2i_sim.pkl', 'rb'))
#     else:
#         item_w2v_emb_dict = trian_item_word2vec(all_click_df)    
#     return item_content_emb_dict, item_w2v_emb_dict, item_youtube_emb_dict, user_youtube_emb_dict
    return item_content_emb_dict

### S7.读取文章信息

In [29]:
def get_article_info_df():
    article_info_df = pd.read_csv(data_path + 'articles.csv')
    article_info_df = reduce_mem(article_info_df)
    return article_info_df

### S8.处理得到的数据

In [30]:
# 这里offline的online的区别就是验证集是否为空
# 调用S4编写的真正的训练、测试、验证集函数
click_trn, click_val, click_tst, val_ans = get_trn_val_tst_data(data_path, offline=False)
click_trn_hist, click_trn_last = get_hist_and_last_click(click_trn)
if click_val is not None:  # 如果click_val就是点击验证集非空，说明是offline模式
    click_val_hist, click_val_last = click_val, val_ans
else:                       # 如果click_val就是点击验证集为空，说明是online模式
    click_val_hist, click_val_last = None, None  
click_tst_hist = click_tst

-- Mem. usage decreased to 21.99 Mb (75.0% reduction),time spend:0.00 min


In [31]:
click_trn_last

,user_id,click_article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type
0,0,157507,1.510000e+12,4,1,17,1,25,2
1,1,289197,1.510000e+12,4,1,17,1,25,6
2,2,168401,1.510000e+12,4,3,20,1,25,2
3,3,50644,1.510000e+12,4,3,2,1,25,2
4,4,42567,1.510000e+12,4,1,12,1,16,1
...,...,...,...,...,...,...,...,...,...
196109,199995,336476,1.510000e+12,4,1,17,1,25,2
196110,199996,299697,1.510000e+12,4,1,17,1,25,2
196111,199997,224171,1.510000e+12,4,1,17,1,16,1
196112,199998,336250,1.510000e+12,4,1,17,1,25,5


- click_trn    ：点击训练数据集 
- click_val    ：线下模式的验证集——这里为空，因为是online
- click_tst    ：点击训练测试集
- val_ans     ：线下模式做验证的答案结果集——这里为空，因为是online
- click_trn_hist：点击历史训练数据集
- click_trn_last：点击末次训练数据集
- click_val_hist：线上模式历史训练数据 
- click_val_last：线上模式末次训练数据
- click_tst_hist：点击历史测试数据集

### S9.读取之前计算好的召回列表
——这里选取的是ItemCF召回

In [32]:
recall_list_dict = get_recall_list(save_path, single_recall_model='i2i_itemcf')
# recall_list_dict = pickle.load(open(save_path + 'final_recall_items_dict.pkl', 'rb')) #两种读取的返回结果有点儿区别

- 为了方便起见，将召回列表字典改成Dataframe形式：

In [33]:
def change_recall_list_df(recall_list_dict):
    df_row_list = [] # [user, item, score]
    for user, recall_list in tqdm(recall_list_dict.items()):
        for item, score in recall_list:
            df_row_list.append([user, item, score])
    column_names = ['user_id', 'sim_item', 'score']
    recall_list_df = pd.DataFrame(df_row_list, columns=column_names)
    return recall_list_df
recall_list_df = change_recall_list_df(recall_list_dict)
# 小处理一下召回数据集
need_append_info_fortag = pd.DataFrame(click_trn_last.iloc[:,:2])
need_append_info_fortag.columns = ["user_id","sim_item"]
need_append_info_fortag["score"] = 1.63513147 
need_append_info_fortag["score"] = need_append_info_fortag["score"].apply(lambda x:x + (np.random.random(1)/1000)[0])
recall_list_df=recall_list_df.append(need_append_info_fortag)
recall_list_df=recall_list_df.sort_values(["user_id","score"])
# 恢复索引
recall_list_df.index = range(recall_list_df.shape[0])

100%|███████████████████████████████████████████████████████████████████████| 247496/247496 [00:05<00:00, 48328.79it/s]


# C2. 特征工程环节之一——负采样处理；

## P1.召回数据需要打Label；——转换成监督学习的前提；
在click_hr_last中做些处理，结合物品相似度进行join；
但是在该场景的情况下，正负样本明显失衡，需要进行一些处理；
*** 很关键的一步骤：打标签；
- 函数get_rank_label_df()；传入参数：
> 1. recall_list_df：已经处理成Dataframe形式的召回列表：user_id,item,score...
> 2. label_df：这里其实是click_trn_last，就是每个用户最后一次点击的信息，如果和上述recall_list_df以[user_id，item]联合关联，有结果的话，说明召回有过点击（本案例是最后一次所以要去单独拿出click_trn_last做最后一次区分；），为正样本；否则就是负样本，未曾点击；
> 3. 最后返回user_id,sim_item,score,label形式的dataframe；
> 4. 就是在recall_list_df的基础上去添加了正负样本标签而已；

In [15]:
def get_rank_label_df(recall_list_df, label_df, is_test=False):
    '''
    传入参数：
    recall_list_df：已经处理成Dataframe形式的召回列表：user_id,item,score...
    label_df：这里其实是click_trn_last，就是每个用户最后一次点击的信息，如果和上述recall_list_df以[user_id，item]联合关联，
              有结果的话，说明召回有过点击（本案例是最后一次所以要去单独拿出click_trn_last做最后一次区分；），为正样本；
              否则就是负样本，未曾点击；
    '''
    # 测试集是没有标签了，为了后面代码同一一些，先给一个默认值-1
    if is_test:
        recall_list_df['label'] = -1
        return recall_list_df
    label_df = label_df.rename(columns={'click_article_id': 'sim_item'}) # 将点击文章click_article_id重命名成sim_item
    # 在召回列表df上拼接新的信息，用户点击时间，点击文章，用户ID，拼接对象是每个用户的末次点击训练集；
    # 作为标签儿判断，click_timestamp为空就是0，1就是点击过
    recall_list_df_ = recall_list_df.merge(label_df[['user_id', 'sim_item', 'click_timestamp']], how='left', on=['user_id', 'sim_item'])
    # 0 表示末次未点，1表示末次点了；
    recall_list_df_['label'] = recall_list_df_['click_timestamp'].apply(lambda x: 0.0 if np.isnan(x) else 1.0)
    # 把不用的列删除掉！
    del recall_list_df_['click_timestamp']
    return recall_list_df_
recall_user_item_label_df = get_rank_label_df(recall_list_df, click_trn_last)
recall_user_item_label_df.head()

,user_id,sim_item,score,label
0,0,211442,0.284456,0.0
1,0,299499,0.297558,0.0
2,0,285869,0.313568,0.0
3,0,203336,0.352849,0.0
4,0,284220,0.368981,0.0


In [16]:
recall_user_item_label_df["label"].value_counts()

0.0    4949920
1.0     197496
Name: label, dtype: int64

## P2.负采样函数——后续要用；

- 函数：neg_sample_recall_data()
- 在刚才已经打过标签的recall_user_item_label_df基础上进行负采样；
- 目的是：在每个用户的带有标签的召回信息中，负样本数，也就是未点击的用户-物品关联至少有1，最多有5个；分别从用户+物品的角度去处理
- 最后合并，再去重

In [17]:
def neg_sample_recall_data(recall_user_item_label_df, sample_rate=0.001): #采样处理的比例
    pos_data = recall_user_item_label_df[recall_user_item_label_df['label'] == 1] ### 正样本标签
    neg_data = recall_user_item_label_df[recall_user_item_label_df['label'] == 0] ### 负样本标签
    # 临时打印一下正负样本的相互比例；
    print('pos_data_num:', len(pos_data), 'neg_data_num:', len(neg_data), 'pos/neg:', len(pos_data)/len(neg_data)) 
    # 分组采样函数，防止出现没有负样本出现的情况；
    # 其实是对label为0的召回信息+标签部分进行处理；
    def neg_sample_func(neg_part_group_df):
        neg_num = len(neg_part_group_df) 
        sample_num = max(int(neg_num * sample_rate), 1) # 保证最少有一个
        sample_num = min(sample_num, 5) # 保证最多不超过5个，这里可以根据实际情况进行选择
        return neg_part_group_df.sample(n=sample_num, replace=True)
    # 对用户进行负采样，保证所有用户都在采样后的数据中
    neg_data_user_sample = neg_data.groupby('user_id', group_keys=False).apply(neg_sample_func) #group_keys=False防止篡改原来索引
    # 对文章进行负采样，保证所有文章都在采样后的数据中
    neg_data_item_sample = neg_data.groupby('sim_item', group_keys=False).apply(neg_sample_func)
    # 将上述两种情况下的采样数据合并
    neg_data_new = neg_data_user_sample.append(neg_data_item_sample)
    # 由于上述两个操作是分开的，可能将两个相同的数据给重复选择了，所以需要对合并后的数据进行去重
    neg_data_new = neg_data_new.sort_values(['user_id', 'score']).drop_duplicates(['user_id', 'sim_item'], keep='last')
    # 将正样本数据合并
    data_new = pd.concat([pos_data, neg_data_new], ignore_index=True)   
    return data_new

## P3.通过召回列表，点击历史，验证历史，测试历史，末次点击训练
- 函数：get_user_recall_item_label_df()：
> 1. 将上述的打标签函数 + 负采样函数结合到一起；
> 2. 同时对测试集中的用户也进行标签操作；
> 3. 最终得到
>> 1. S1. trn_user_item_label_df：训练用的样本，用户召回物品标签
>> 2. S2. val_user_item_label_df：验证用的样本，用户召回物品标签——这里为空；
>> 3. S3. tst_user_item_label_df：测试用的样本，用户召回物品标签

In [18]:
trn_user_items_df = recall_list_df[recall_list_df['user_id'].isin(click_trn_hist['user_id'].unique())]
trn_user_items_df.shape

(4147416, 3)

In [19]:
# 打标签环节：
trn_user_item_label_df = get_rank_label_df(trn_user_items_df, click_trn_last, is_test=False)
trn_user_item_label_df.shape,trn_user_item_label_df["label"].value_counts()

((4147416, 4),
 0.0    3949920
 1.0     197496
 Name: label, dtype: int64)

In [20]:
# 负采样处理：
trn_user_item_label_df = neg_sample_recall_data(trn_user_item_label_df)
trn_user_item_label_df.shape,trn_user_item_label_df["label"].value_counts()

pos_data_num: 197496 neg_data_num: 3949920 pos/neg: 0.05


((410496, 4),
 0.0    213000
 1.0    197496
 Name: label, dtype: int64)

In [21]:
# 可减负样本数量比例发生了变化！样本间更见平衡一些。。。

In [22]:
def get_user_recall_item_label_df(click_trn_hist, click_val_hist, click_tst_hist,click_trn_last, click_val_last, recall_list_df):
    # 获取训练数据的召回列表,就是——从训练点击历史集的所有用户他们的召回列表信息，当然不能用click_trn_last。。。人家是用来打标签的
    trn_user_items_df = recall_list_df[recall_list_df['user_id'].isin(click_trn_hist['user_id'].unique())]
    # 训练数据打标签
    trn_user_item_label_df = get_rank_label_df(trn_user_items_df, click_trn_last, is_test=False)
    # 训练数据负采样
    trn_user_item_label_df = neg_sample_recall_data(trn_user_item_label_df)
    
    if click_val is not None:
        val_user_items_df = recall_list_df[recall_list_df['user_id'].isin(click_val_hist['user_id'].unique())]
        val_user_item_label_df = get_rank_label_df(val_user_items_df, click_val_last, is_test=False)
        val_user_item_label_df = neg_sample_recall_data(val_user_item_label_df)
    # 走的是下面的这一段；所以val_user_item_label_df为空
    else:
        val_user_item_label_df = None
    # 测试数据不需要进行负采样，直接对所有的召回商品进行打-1标签
    tst_user_items_df = recall_list_df[recall_list_df['user_id'].isin(click_tst_hist['user_id'].unique())]
    tst_user_item_label_df = get_rank_label_df(tst_user_items_df, None, is_test=True)
    return trn_user_item_label_df, val_user_item_label_df, tst_user_item_label_df

In [23]:
# 给训练验证数据打标签，并负采样（这一部分时间比较久）
trn_user_item_label_df, val_user_item_label_df, tst_user_item_label_df = get_user_recall_item_label_df(click_trn_hist, 
                                                                                                       click_val_hist, 
                                                                                                       click_tst_hist,
                                                                                                       click_trn_last, 
                                                                                                       click_val_last, 
                                                                                                       recall_list_df)


pos_data_num: 197496 neg_data_num: 3949920 pos/neg: 0.05


In [24]:
trn_user_item_label_df.shape,val_user_item_label_df,tst_user_item_label_df.shape

((410486, 4), None, (1000000, 4))

## P4.最终的召回做字典存储；

In [25]:
## 将最终的召回的df数据转换成字典的形式做排序特征
def make_tuple_func(group_df):
    row_data = []
    # 遍历上面三个集合，把物品，分数，标签拿过来；分数做以下处理太长了
    for name, row_df in group_df.iterrows():
        row_data.append((row_df['sim_item'], round(row_df['score'],6), row_df['label']))
    return row_data

In [26]:
# 训练集用的部分；以user为索引groupby后再复原勿忘
trn_user_item_label_tuples = trn_user_item_label_df.groupby('user_id').apply(make_tuple_func).reset_index()
trn_user_item_label_tuples_dict = dict(zip(trn_user_item_label_tuples['user_id'], trn_user_item_label_tuples[0]))

In [27]:
# 验证集和测试集同样的处理法则；
if val_user_item_label_df is not None:
    val_user_item_label_tuples = val_user_item_label_df.groupby('user_id').apply(make_tuple_func).reset_index()
    val_user_item_label_tuples_dict = dict(zip(val_user_item_label_tuples['user_id'], val_user_item_label_tuples[0]))
else:
    val_user_item_label_tuples_dict = None
    
tst_user_item_label_tuples = tst_user_item_label_df.groupby('user_id').apply(make_tuple_func).reset_index()
tst_user_item_label_tuples_dict = dict(zip(tst_user_item_label_tuples['user_id'], tst_user_item_label_tuples[0]))


In [32]:
# trn_user_item_label_tuples_dict,tst_user_item_label_tuples_dict

# C3.特征工程环节之二——用户历史行为，结合历史点击过的文章和召回的文章创造一些相似度上+词数+时间上的特征；
- 因为在浏览新闻的环节中，用户对于文章长短有偏少词数的倾向；并且新闻浏览的时效性比传统物品的主题浏览的偏好性会更强；所以看一下相似度词数创建时间等特征信息；
- 再一个很关键的环节；
- 函数create_feature()
- 前提之一：调用上面编写好的函数get_article_info_df()获取文章的信息article_info_df；
- 前提之二：click_trn_hist，click_tst_hist；
- 前提之三：上述已创建好的trn_user_item_label_tuples_dict,tst_user_item_label_tuples_dict；
- 物品embedding信息，get_embedding()函数，为了计算召回物品与最终点击物品的相似度；
- 针对上述划分好的click_trn_hist，click_tst_hist，上述已创建好的trn_user_item_label_tuples_dict,tst_user_item_label_tuples_dict；
分别做特征创造；
- 目的梳理清楚；针对每一个用户遍历，在其召回物品列表，分数，标签值的基础上，结合其点击历史信息；
> 1. 遍历每一个用户：从其召回列表信息中，即文章物品——分数——标签；拿到该文章物品的时间，词数等信息；
并生成该用户每个召回文章物品的列表信息[user_id, item]——后续基础信息结构搭建；
> 2. 遍历该用户访问的最后N次的文章物品（通常是最后一次），分别计算与召回文章物品的相似度，创建时间，词数差等信息；
追加加到[user_id, item]后面中，并添加相似度的最大最小平均中位值等额外信息，打分，标签不能丢；
> 3.所有的信息添加到最终的all_user_feas做Dataframe处理
- 最后的形式：user_id、click_article_id、sim0、time_diff0、word_diff0、sim_max、sim_min、sim_sum、sim_mean、sim_median、score、rank、label
- 得到的字段解释：
sim0：      召回的文章物品与最后一次点击的文章之间的相似度；
time_diff0：  召回的文章物品与最后一次点击的文章之间创建时间的差值；
word_diff0：  召回的文章物品与最后一次点击的文章之间文章词数的差值；
sim_max：    用户层面上召回的所有文章物品，与最后一次点击的文章之间的相似度最大值；
sim_min：    用户层面上召回的所有文章物品，与最后一次点击的文章之间的相似度最小值；
sim_sum：    用户层面上召回的所有文章物品，与最后一次点击的文章之间的相似度的Σ值；
sim_mean：   用户层面上召回的所有文章物品，与最后一次点击的文章之间的相似度平均值；
sim_median：  用户层面上召回的所有文章物品，与最后一次点击的文章之间的相似度中位值；
score：     召回的文章物品分数；
rank：      召回的文章物品与最后一次点击的文章之间的排序；
label：     是否是用户最后一次点击；——本项目中的特殊性，因为要求是最后一次点击所以要用最后一次click_trn_hist找到最后一次拼接标签；
——如果是正常情况只需要召回的文章物品中看点没点就行；

## P1.读取文章向量和文章基本属性信息 

In [34]:
article_info_df = get_article_info_df()
articles_emb = get_embedding(save_path, all_click_df)

-- Mem. usage decreased to  0.25 Mb (50.0% reduction),time spend:0.00 min


In [29]:
# click_trn_hist[click_trn_hist["user_id"]==163549]
# trn_user_item_label_tuples_dict[163549]
# # 建立一个二维列表保存结果， 后面要转成DataFrame，这里选取任意一个用户看一下流程,163549这个用户和训练集trn_user_item_label_tuples_dict
# all_user_feas = []
# i = 0
# # 获取了该用户的最后一次点击的物品，该用户最后一次点击文章是211732
# hist_user_items_163549 = click_trn_hist[click_trn_hist['user_id']==163549]['click_article_id'][-1:]
# # 遍历该用户的召回列表，即召回文章、分数、标签：该用户召回是285663+205866+202534三篇
# for rank, (article_id, score, label) in enumerate(trn_user_item_label_tuples_dict[163549]):
#     # 召回的文章物品创建时间
#     recall_items_create_time = article_info_df[article_info_df['article_id']==article_id]['created_at_ts'].values[0] 
#     # 召回的文章物品词数信息
#     recall_items_words_count = article_info_df[article_info_df['article_id']==article_id]['words_count'].values[0]
#     single_user_fea = [163549, article_id]
#     # 计算与最后点击的商品的相似度的和， 最大值和最小值， 均值    
#     sim_fea = []                                     # sim_fea：相似度特征
#     time_fea = []                                    # time_fea：时间特征
#     words_fea = []                                   # word_fea词数差特征
#     # 遍历用户的最后N次点击文章，这里是最后一次,比较一下召回的文章物品在创建时间、字数、相似度上的差距，所以就1个
#     for hist_item in hist_user_items_163549:
#         b_create_time = article_info_df[article_info_df['article_id']==hist_item]['created_at_ts']. values[0]
#         b_words_count = article_info_df[article_info_df['article_id']==hist_item]['words_count'].values[0]
#         sim_fea.append(cosine_similarity([articles_emb[hist_item],articles_emb[article_id]])[0][1])            # 计算召回文章与末N次点击文章的相似度
#         time_fea.append(abs(recall_items_create_time-b_create_time))                                           # 计算召回文章与末N次点击文章的时间差
#         words_fea.append(abs(recall_items_words_count-b_words_count))                                          # 计算召回文章与末N次点击文章的词数差
#     single_user_fea.extend(sim_fea)      # 相似性特征
#     single_user_fea.extend(time_fea)    # 时间差特征
#     single_user_fea.extend(words_fea)    # 字数差特征
#     single_user_fea.extend([max(sim_fea), min(sim_fea), sum(sim_fea), sum(sim_fea) / len(sim_fea),np.median(sim_fea)])  # 相似性的统计特征
#     single_user_fea.extend([score, rank, label])      
#     all_user_feas.append(single_user_fea)
# all_user_feas

## P2.下面基于data做历史相关的特征，这里去拿最后1次点击；

In [8]:
# 定义历史特征相关计算函数
def create_feature(users_id, recall_list, click_hist_df,  articles_info, articles_emb, user_emb=None, N=1):
    """
    基于用户的历史行为做相关特征
    : users_id: 用户id
    : recall_list: 对于每个用户召回的候选文章列表，候选值：trn_user_item_label_tuples_dict&tst_user_item_label_tuples_dict
    : click_hist_df: 用户的历史点击信息，候选值：click_trn_hist&click_tst_hist
    : articles_info: 文章信息
    : articles_emb: 文章的embedding向量, 这个可以用item_content_emb, item_w2v_emb, item_youtube_emb
    :param N: 最近的N次点击  由于testA日志里面很多用户只存在一次历史点击， 所以为了不产生空值，默认是1
    """
    # 建立一个二维列表保存结果， 后面要转成DataFrame
    all_user_feas = []
    i = 0
    for user_id in tqdm(users_id):
        # 该用户的最后N次点击，这里是最后1次采用；
        hist_user_items = click_hist_df[click_hist_df['user_id']==user_id]['click_article_id'][-N:]
        # 遍历该用户的召回列表
        for rank, (article_id, score, label) in enumerate(recall_list[user_id]):
            # 该文章建立时间, 字数
            recall_item_create_time = articles_info[articles_info['article_id']==article_id]['created_at_ts'].values[0]
            recall_item_words_count = articles_info[articles_info['article_id']==article_id]['words_count'].values[0]
            single_user_fea = [user_id, article_id]
            # 计算与最后点击的商品的相似度，聚合运算其相似度的和， 最大值和最小值， 均值，中位值
            sim_fea = []
            time_fea = []
            words_fea = []
            # 遍历用户的最后N次点击文章
            for hist_item in hist_user_items:
                final_event_create_time = articles_info[articles_info['article_id']==hist_item]['created_at_ts'].values[0]
                final_event_words_count = articles_info[articles_info['article_id']==hist_item]['words_count'].values[0]
                sim_fea.append(np.dot(articles_emb[hist_item], articles_emb[article_id]))
                time_fea.append(abs(recall_item_create_time-final_event_create_time))
                words_fea.append(abs(recall_item_words_count-final_event_words_count))
            single_user_fea.extend(sim_fea)      # 相似性特征
            single_user_fea.extend(time_fea)    # 时间差特征
            single_user_fea.extend(words_fea)    # 字数差特征
            single_user_fea.extend([max(sim_fea), min(sim_fea), sum(sim_fea), sum(sim_fea) / len(sim_fea), np.median(sim_fea)])  # 相似性的统计特征
            single_user_fea.extend([score, rank, label])    
            # 加入到总的表中
            all_user_feas.append(single_user_fea)
    # 定义列名
    id_cols = ['user_id', 'click_article_id']
    sim_cols = ['sim' + str(i) for i in range(N)]
    time_cols = ['time_diff' + str(i) for i in range(N)]
    word_cols = ['word_diff' + str(i) for i in range(N)]
    sat_cols = ['sim_max', 'sim_min', 'sim_sum', 'sim_mean' , 'sim_median']
    user_item_sim_cols = ['user_item_sim'] if user_emb else []
    user_score_rank_label = ['score', 'rank', 'label']
    cols = id_cols + sim_cols + time_cols + word_cols + sat_cols + user_item_sim_cols + user_score_rank_label
    # 转成DataFrame
    df = pd.DataFrame(all_user_feas, columns=cols)
    return df


In [31]:
# user_163549_info = create_feature([163549], trn_user_item_label_tuples_dict, click_trn_hist, article_info_df,articles_emb, user_emb=None, N=3)

## P3. 获取训练验证及测试数据中召回列文章相关特征

In [33]:
trn_user_item_feats_df = create_feature(trn_user_item_label_tuples_dict.keys(), trn_user_item_label_tuples_dict, \
                                            click_trn_hist, article_info_df, articles_emb)
if val_user_item_label_tuples_dict is not None:
    val_user_item_feats_df = create_feature(val_user_item_label_tuples_dict.keys(), val_user_item_label_tuples_dict, \
                                                click_val_hist, article_info_df, articles_emb)
else:
    val_user_item_feats_df = None
tst_user_item_feats_df = create_feature(tst_user_item_label_tuples_dict.keys(), tst_user_item_label_tuples_dict, \
                                            click_tst_hist, article_info_df, articles_emb)

100%|████████████████████████████████████████████████████████████████████████████| 50000/50000 [28:58<00:00, 28.77it/s]


- 如果未生成新的特征工程处理的数据，需要运行以下代码：

In [35]:
# # 保存一份省的每次都要重新跑，每次跑的时间都比较长
# trn_user_item_feats_df.to_csv(save_dir + 'trn_user_item_feats_df.csv', index=False)

# if val_user_item_feats_df is not None:
#     val_user_item_feats_df.to_csv(save_dir + 'val_user_item_feats_df.csv', index=False)

# tst_user_item_feats_df.to_csv(save_dir + 'tst_user_item_feats_df.csv', index=False)    

- 如果已生成新的特征工程处理的数据，需要运行以下代码：

In [ ]:
trn_user_item_feats_df = pd.read_csv(save_dir + 'trn_user_item_feats_df.csv')
tst_user_item_feats_df = pd.read_csv(save_dir + 'tst_user_item_feats_df.csv')

# C4.特征工程环节之三——用户/文章特征构造处理；

- 有几个方面可以考虑：
> 1. 用户点击文章次数+点击时间构造，可以表现用户活跃度：前面召回阶段已经有过初步判断；
> 2. 文章物品被点击次数+被点击时间构造可以反映文章热度的特征；
> 3. 用户的浏览时间偏好性： 根据其点击的历史文章列表的点击时间和文章的创建时间做统计特征，比如求均值， 这个可以反映用户对于文章时效的偏好；
> 4. 用户的主题偏好性， 对于用户点击的历史文章主题进行一个统计， 然后对于当前文章看看是否属于用户已经点击过的主题；
> 5. 用户的字数偏好性特征， 对于用户点击的历史文章的字数统计， 求一个均值，看看用户喜欢长文短文；

## P0.准备阶段——文章和用户信息整合：
- all_data：用户点击+文章的信息联合df

In [ ]:
# article_info_df前面已经读取完毕，进行一下内存减少处理，更名articles
articles = reduce_mem(article_info_df)
# 训练集测试集一起
if click_val is not None:
    all_data = click_trn.append(click_val)
all_data = click_trn.append(click_tst)
# 加入文章信息，注意click_article_id和article_id两列相同；
all_data = all_data.merge(articles, left_on='click_article_id', right_on='article_id')
all_data = reduce_mem(all_data)
all_data.head()

## P1.用户活跃度的判断；
- 分析一下每个用户点击时间和点击文章的次数，区分用户活跃度；
> 1. 用户点击行为之间时间间隔差距小，并且点击次数多，说明活跃程度高；
> 2. 对于每个用户，计算点击文章的次数， 两两点击文章时间间隔的均值；
> 3. 把点击次数&时间间隔的倒数的均值统一归一化，然后两者相加加和，该值越大，说明用户越活跃；——原版是越小越活跃太难受了调过来！
> 4. 最终得到的一列是user_id,click_size次数的倒数，time_diff_mean平均点击时间间隔，active_level_parameter活跃度参数！
- 最终结果：user_act_fea

In [34]:
# activition_need_cols = ['user_id', 'click_article_id', 'click_timestamp']
# activition_data = all_data[activition_need_cols]
# activition_data.sort_values(['user_id', 'click_timestamp'], inplace=True)
# # all_data[all_data["user_id"]==249999]
# user_act_df = pd.DataFrame(activition_data.groupby('user_id', as_index=False)[['click_article_id', 'click_timestamp']].\
#                             agg({'click_article_id':np.size, 'click_timestamp': {list}}).values, columns=['user_id', 'click_size', 'click_timestamp'])
# def time_diff_mean(l):
#     if len(l) == 1:
#         return 1
#     else:
#         return np.mean([j-i for i, j in list(zip(l[:-1], l[1:]))])
# user_act_df['time_diff_mean'] = user_act_df['click_timestamp'].apply(lambda x: time_diff_mean(x))
# # 点击次数取倒数
# user_act_df['click_size'] = 1 / user_act_df['click_size']
# # 将点击事件间隔均值和点击次数分别归一化
# user_act_df['click_size'] = (user_act_df['click_size'] - user_act_df['click_size'].min()) / (user_act_df['click_size'].max() - user_act_df['click_size'].min())
# user_act_df['time_diff_mean'] = (user_act_df['time_diff_mean'] - user_act_df['time_diff_mean'].min()) / (user_act_df['time_diff_mean'].max() - user_act_df['time_diff_mean'].min())     
# # 新增 用户活跃参数：active_level_parameter，再去取倒数，表示越高越活跃
# user_act_df['active_level_parameter'] = 1/(user_act_df['click_size'] + user_act_df['time_diff_mean'])
# user_act_df['user_id'] = user_act_df['user_id'].astype('int')
# del user_act_df['click_timestamp']
# user_act_df
#################################￥￥￥￥￥￥￥￥￥￥￥￥￥￥￥￥￥￥￥￥￥￥￥￥￥￥￥￥￥￥￥
# 活跃度计算参数的函数！
def user_activition_level(all_data, activition_need_cols):
    """
    用户活跃度参数的计算流程函数：
    :all_data: 全数据集
    :activition_need_cols: 计算活跃度用到的特征列，这里是点击的次数：文章的ID（做count）和点击的时间（为了做差）！
    """
    user_activition_data = all_data[activition_need_cols]
    # 接下来要对每一个用户，每一次点击时间排序；
    user_activition_data.sort_values(['user_id', 'click_timestamp'], inplace=True)
    # 对每一个用户groupby，统计其点击次数，以及每次点击的时间，groupby到一个格子里,agg指定统计后的形式，一个数字，一个列表存储；
    user_act_df = pd.DataFrame(user_activition_data.groupby('user_id', as_index=False)[['click_article_id', 'click_timestamp']].\
                            agg({'click_article_id':np.size, 'click_timestamp': {list}}).values, columns=['user_id', 'click_size', 'click_timestamp'])
    # 计算时间间隔的均值
    def time_diff_mean(l):
        if len(l) == 1:
            return 1
        # 分别计算相邻两次的时间差
        # :-1表示除了最后一个往前的所有元素；1:表示除了第一个往后的所有元素；所有的时间差计算以后取平均值
        else:
            return np.mean([j-i for i, j in list(zip(l[:-1], l[1:]))])
    user_act_df['time_diff_mean'] = user_act_df['click_timestamp'].apply(lambda x: time_diff_mean(x))
    # 点击次数取到数
    user_act_df['click_size'] = 1/user_act_df['click_size']
    # 将点击事件间隔均值和点击次数分别归一化
    user_act_df['click_size'] = (user_act_df['click_size'] - user_act_df['click_size'].min()) / (user_act_df['click_size'].max() - user_act_df['click_size'].min())
    user_act_df['time_diff_mean'] = (user_act_df['time_diff_mean'] - user_act_df['time_diff_mean'].min()) / (user_act_df['time_diff_mean'].max() - user_act_df['time_diff_mean'].min())     
    # 新增 用户活跃参数：active_level_parameter，倒过来，越大越活跃表示，
    user_act_df['active_level_parameter'] = 1 /(user_act_df['click_size'] + user_act_df['time_diff_mean'] )
    user_act_df['user_id'] = user_act_df['user_id'].astype('int')
    del user_act_df['click_timestamp']
    return user_act_df

- 将用户活跃度结果计算出来：user_act_fea

In [35]:
activition_need_cols = ['user_id', 'click_article_id', 'click_timestamp']
user_act_fea = user_activition_level(all_data, activition_need_cols)

## P2.分析一下文章的点击时间和被点击次数， 衡量文章热度特征；
- 分析一下文章被点击时间和被点击次数，区分文章热门程度；
> 1. 根据文章进行分组， 计算点击的时间间隔平均值；
> 2. 计算文章被点击的次数；
> 3. 将文章点击的用户数量取倒数，并将用户数量和时间间隔归一化，相加得到文章的热度特征再取倒数，该值越大，说明文章越热；
> 4. 最终得到的一列是click_article_id,user_num被点击次数的倒数，time_diff_mean平均点击时间间隔，hot_parameter活跃度参数！
- 最终结果：articles_hot_fea

In [37]:
def articles_hot_level(all_data, hot_need_cols):
    """
    文章热门程度的
    :all_data: 全数据集
    :activition_need_cols: 计算文章热度用到的特征列，这里是点击的次数：文章的ID（做count）和点击的时间（为了做差）！
    """
    articles_hot_data = all_data[hot_need_cols]
    # 接下来要对每一篇文章，按每一次点击时间排序；
    articles_hot_data.sort_values(['click_article_id', 'click_timestamp'], inplace=True)
    # 对每一篇文章groupby，统计其被点击次数，以及每次点击的时间，groupby到一个格子里,agg指定统计后的形式，一个数字，一个列表存储；
    article_hot_df = pd.DataFrame(articles_hot_data.groupby('click_article_id', as_index=False)[['user_id', 'click_timestamp']].\
                               agg({'user_id':np.size, 'click_timestamp': {list}}).values, columns=['click_article_id', 'user_num', 'click_timestamp'])
    # 计算被点击时间间隔的均值
    def time_diff_mean(l):
        if len(l) == 1:
            return 1
        else:
            return np.mean([j-i for i, j in list(zip(l[:-1], l[1:]))])
    article_hot_df['time_diff_mean'] = article_hot_df['click_timestamp'].apply(lambda x: time_diff_mean(x))
    # 被点击次数取倒数
    article_hot_df['user_num'] = 1 / article_hot_df['user_num']
    # 两者归一化
    article_hot_df['user_num'] = (article_hot_df['user_num'] - article_hot_df['user_num'].min()) / (article_hot_df['user_num'].max() - article_hot_df['user_num'].min())
    article_hot_df['time_diff_mean'] = (article_hot_df['time_diff_mean'] - article_hot_df['time_diff_mean'].min()) / (article_hot_df['time_diff_mean'].max() - article_hot_df['time_diff_mean'].min())     
    # 新增 文章热度参数：hot_parameter，倒过来，越大越活跃表示
    article_hot_df['hot_parameter'] = 1 /(article_hot_df['user_num'] + article_hot_df['time_diff_mean'])
    article_hot_df['click_article_id'] = article_hot_df['click_article_id'].astype('int')
    del article_hot_df['click_timestamp']
    return article_hot_df

- 将文章热门度结果计算出来：articles_hot_fea

In [38]:
hot_need_cols = ['user_id', 'click_article_id', 'click_timestamp']
articles_hot_fea = articles_hot_level(all_data, hot_need_cols)

# C5.特征工程环节之四——用户习惯偏好层面的数据挖掘；

- 从all_click_df就是全部点击日志中，可以看到点击环境，时间，设备，地区，类型等信息；这个是不是可以视为所谓的用户画像？
- 1.用户的设备习惯， 这里取最常用的设备，最多的是哪个——取众数；
- 2.用户的时间习惯： 用户习惯啥时候看新闻和物品；
- 3.用户的爱好特征， 文章的类别、主题信息分类；
- 4.用户文章的词数， 用户的爱好文章的字数习惯，偏爱短文章长文章；

## P1.用户设备习惯特征挖掘
- 将设备，环境，地区等信息分别统计出每个用户最多的那一个！
- 返回user_device_info_df

In [39]:
# def device_fea(all_data, cols):
#     """
#     用户的最习惯设备特征类型；
#     :all_data: 数据集
#     :cols: 用到的特征列
#     """
#     user_device_info = all_data[cols]
    
#     # 用众数来表示每个用户的设备信息
#     user_device_info = user_device_info.groupby('user_id').agg(lambda x: x.value_counts().index[0]).reset_index()
    
#     return user_device_info
    
# # 设备特征(这里时间会比较长)
# device_cols = ['user_id', 'click_environment', 'click_deviceGroup', 'click_os', 'click_country', 'click_region', 'click_referrer_type']
# user_device_info = device_fea(all_data, device_cols)
###################################################################￥￥￥￥￥￥￥￥￥￥￥￥￥￥￥%%%%%%%%%%%%%%%%%%%
# 定义设备习惯需要的列
device_need_cols = ['user_id', 'click_environment', 'click_deviceGroup', 'click_os', 'click_country', 'click_region', 'click_referrer_type']
# 指定列后统计列上每一特征的出现“种类数”；从每一个特征中挑选出出现次数最多的！
user_device_info = all_data[device_need_cols]
user_device_info_df = user_device_info.groupby('user_id').agg(lambda x: x.value_counts().index[0]).reset_index()

## P2.用户时间习惯特征挖掘
- 个人意见是时间戳转换成时间格式，其实换成周几+几点！
- 返回user_timehobby_info：用户id-点击周几-点击几点-文章创建周几-文章创建几点

In [40]:
# def user_time_hob_fea(all_data, cols):
#     """
#     制作用户的时间习惯特征
#     :param all_data: 数据集
#     :param cols: 用到的特征列
#     """
#     user_time_hob_info = all_data[cols]
#     # 先把时间戳进行归一化
#     mm = MinMaxScaler()
#     user_time_hob_info['click_timestamp'] = mm.fit_transform(user_time_hob_info[['click_timestamp']])
#     user_time_hob_info['created_at_ts'] = mm.fit_transform(user_time_hob_info[['created_at_ts']])
#     user_time_hob_info = user_time_hob_info.groupby('user_id').agg('mean').reset_index()
#     user_time_hob_info.rename(columns={'click_timestamp': 'user_time_hob1', 'created_at_ts': 'user_time_hob2'}, inplace=True)
#     return user_time_hob_info
# user_time_hob_cols = ['user_id', 'click_timestamp', 'created_at_ts']
# user_time_hob_info = user_time_hob_fea(all_data, user_time_hob_cols)
user_timehobby_need_cols = ['user_id', 'click_timestamp', 'created_at_ts']
user_timehobby_info = all_data[user_timehobby_need_cols]
# 单独生成一列，是点击文章在星期几
user_timehobby_info["click_timestamp_1"] = user_timehobby_info["click_timestamp"].astype("int64").apply(lambda x: pd.to_datetime(x,unit='ms').weekday()+1)
# 再生成一列，是点击文章在几点
user_timehobby_info["click_timestamp_2"] = user_timehobby_info["click_timestamp"].astype("int64").apply(lambda x: int(datetime.strftime(pd.to_datetime(x,unit='ms'),"%H:%M:%S")[:2]))
# 文章的创建时间也做同样的两个处理：
user_timehobby_info["created_at_ts_1"] = user_timehobby_info["created_at_ts"].apply(lambda x: pd.to_datetime(x,unit='ms').weekday()+1)
user_timehobby_info["created_at_ts_2"] = user_timehobby_info["created_at_ts"].apply(lambda x: int(datetime.strftime(pd.to_datetime(x,unit='ms'),"%H:%M:%S")[:2]))
del user_timehobby_info["click_timestamp"]
del user_timehobby_info["created_at_ts"]
user_timehobby_info.columns = ["user_id", "click_weekday", "click_hour", "article_crt_weekday", "article_crt_hour"]
# 找出用户最爱在周几点击，几点点击，最喜欢周几创建的文章，几点创建的文章
user_timehobby_info_df = user_timehobby_info.groupby(["user_id"]).agg(lambda x: x.value_counts().index[0]).reset_index()

## P3.用户主题类别偏好特征挖掘
- 把每个用户的点击过得文章类别，变成一列列表特征；
- 返回user_category_hobby_info_df

In [41]:
# def user_cat_hob_fea(all_data, cols):
#     """
#     用户的主题爱好
#     :param all_data: 数据集
#     :param cols: 用到的特征列
#     """
#     user_category_hob_info = all_data[cols]
#     user_category_hob_info = user_category_hob_info.groupby('user_id').agg({list}).reset_index()
#     user_cat_hob_info = pd.DataFrame()
#     user_cat_hob_info['user_id'] = user_category_hob_info['user_id']
#     user_cat_hob_info['cate_list'] = user_category_hob_info['category_id']
#     return user_cat_hob_info
# user_category_hob_cols = ['user_id', 'category_id']
# user_cat_hob_info = user_cat_hob_fea(all_data, user_category_hob_cols)
# 拿出相关列
user_category_need_cols = ['user_id', 'category_id']
user_category_hobby_info = all_data[user_category_need_cols]
# 点击类别列表化
user_category_hobby_info = user_category_hobby_info.groupby('user_id').agg({list}).reset_index()
# 重新Dataframe化
user_category_hobby_info_df = pd.DataFrame()
user_category_hobby_info_df['user_id'] = user_category_hobby_info['user_id']
user_category_hobby_info_df['cate_list'] = user_category_hobby_info['category_id']

## P4.用户的文章词数偏好特征挖掘
- 很简单，用户点击过的文章词语数平均值；其实可以考虑加权重；
- 返回user_wordscount_hobby_info_df

In [42]:
user_wordscount_hobby_info_df = all_data.groupby('user_id')['words_count'].agg('mean').reset_index()
user_wordscount_hobby_info_df.rename(columns={'words_count': 'words_hbo'}, inplace=True)

## P5.用户层面的所有信息汇总；
- 前面处理好的用户活跃度信息：user_act_fea
- 设备偏好：user_device_info_df
- 时间偏好：user_timehobby_info
- 类别偏好：user_category_hobby_info_df
- 词数偏好：user_wordscount_hobby_info_df

In [43]:
user_feature_all_info = pd.merge(user_act_fea, user_device_info_df, on='user_id')
user_feature_all_info = user_feature_all_info.merge(user_timehobby_info_df, on='user_id')
user_feature_all_info = user_feature_all_info.merge(user_category_hobby_info_df, on='user_id')
user_feature_all_info = user_feature_all_info.merge(user_wordscount_hobby_info_df, on='user_id')

## P6.最终用户侧特征保存本地
- 保存到本地指定路径 ：user_feature_info.csv

# C6.最终处理；
- 将前面得到的
>> 1. trn_user_item_feats_df = pd.read_csv(save_dir + 'trn_user_item_feats_df.csv')
>> 2. tst_user_item_feats_df = pd.read_csv(save_dir + 'tst_user_item_feats_df.csv')
- 拼接上
>> 1. user_feature_all_info.to_csv(save_dir + 'user_feature_info.csv', index=False) 
得到trn_user_item_feats_df+tst_user_item_feats_df，并做字段顺序调整；并且对召回文章的类别分析，形成新字段；
即召回文章的主题类别是否在用户的点击过的主题类别里面：即is_cat_hub字段；

### S1.保存/读取用户特征读取；

In [36]:
# 保存好以后就可以读取了
# user_feature_all_info.to_csv(save_dir + 'user_feature_info.csv', index=False)  
# 把用户信息直接读入进来
user_feature_all_info = pd.read_csv(save_dir + 'user_feature_info.csv')

### S2.之前特征划分的训练集+测试集拼上新制作的用户侧特征

In [37]:
# 读取
if os.path.exists(save_dir + 'trn_user_item_feats_df.csv'):
    trn_user_item_feats_df = pd.read_csv(save_dir + 'trn_user_item_feats_df.csv')
    
if os.path.exists(save_dir + 'tst_user_item_feats_df.csv'):
    tst_user_item_feats_df = pd.read_csv(save_dir + 'tst_user_item_feats_df.csv')

if os.path.exists(save_dir + 'val_user_item_feats_df.csv'):
    val_user_item_feats_df = pd.read_csv(save_dir + 'val_user_item_feats_df.csv')
else:
    val_user_item_feats_df = None
# 拼上用户特征
trn_user_item_feats_df = trn_user_item_feats_df.merge(user_feature_all_info, on='user_id', how='left')
if val_user_item_feats_df is not None:
    val_user_item_feats_df = val_user_item_feats_df.merge(user_feature_all_info, on='user_id', how='left')
else:
    val_user_item_feats_df = None
tst_user_item_feats_df = tst_user_item_feats_df.merge(user_feature_all_info, on='user_id',how='left')

### S3.拼接文章物品的特征

In [39]:
# 重新读取也可以
# article_info_df = get_article_info_df()
article_info_df = reduce_mem(article_info_df)
# 拼上文章特征
trn_user_item_feats_df_final = trn_user_item_feats_df.merge(article_info_df, left_on='click_article_id', right_on='article_id')
if val_user_item_feats_df is not None:
    val_user_item_feats_df = val_user_item_feats_df.merge(article_info_df, left_on='click_article_id', right_on='article_id')
else:
    val_user_item_feats_df = None
tst_user_item_feats_df_final = tst_user_item_feats_df.merge(article_info_df, left_on='click_article_id', right_on='article_id') 

-- Mem. usage decreased to  0.25 Mb (0.0% reduction),time spend:0.00 min


In [47]:
trn_user_item_feats_df_final["cate_list_new"] = trn_user_item_feats_df_final["cate_list"].apply(lambda x: set(int(i) for i in x.replace("[","").replace("]","").replace(" ","").split(",")))
tst_user_item_feats_df_final["cate_list_new"] = tst_user_item_feats_df_final["cate_list"].apply(lambda x: set(int(i) for i in x.replace("[","").replace("]","").replace(" ","").split(",")))
trn_user_item_feats_df_final['is_cat_hab'] = trn_user_item_feats_df_final.apply(lambda x: 1 if x.category_id in x.cate_list_new else 0, axis=1)
if val_user_item_feats_df is not None:
    trn_user_item_feats_df_final['is_cat_hab'] = trn_user_item_feats_df_final.apply(lambda x: 1 if x.category_id in set(x.cate_list) else 0, axis=1)
else:
    val_user_item_feats_df = None
tst_user_item_feats_df_final['is_cat_hab'] = tst_user_item_feats_df_final.apply(lambda x: 1 if x.category_id in x.cate_list_new else 0, axis=1)

In [48]:
trn_user_item_feats_df_final.columns

Index(['user_id', 'click_article_id', 'sim0', 'time_diff0', 'word_diff0',
       'sim_max', 'sim_min', 'sim_sum', 'sim_mean', 'sim_median', 'score',
       'rank', 'label', 'click_size', 'time_diff_mean',
       'active_level_parameter', 'click_environment', 'click_deviceGroup',
       'click_os', 'click_country', 'click_region', 'click_referrer_type',
       'click_weekday', 'click_hour', 'article_crt_weekday',
       'article_crt_hour', 'cate_list', 'words_hbo', 'article_id',
       'category_id', 'created_at_ts', 'words_count', 'cate_list_new',
       'is_cat_hab'],
      dtype='object')

## S4.调整一下顺序！

In [51]:
trn_user_item_feats_df_final_sample = trn_user_item_feats_df_final[['user_id', 'click_article_id', 'sim0', 'time_diff0', 'word_diff0',
       'sim_max', 'sim_min', 'sim_sum', 'sim_mean', 'sim_median', 'score','rank', 'click_size', 'time_diff_mean','active_level_parameter', 
       'click_environment', 'click_deviceGroup', 'click_os', 'click_country', 'click_region', 'click_referrer_type', 'click_weekday', 
       'click_hour', 'article_crt_weekday', 'article_crt_hour',  'words_hbo', 'category_id', 'words_count', 'is_cat_hab', 'label']]
tst_user_item_feats_df_final_sample = tst_user_item_feats_df_final[['user_id', 'click_article_id', 'sim0', 'time_diff0', 'word_diff0',
       'sim_max', 'sim_min', 'sim_sum', 'sim_mean', 'sim_median', 'score','rank', 'click_size', 'time_diff_mean','active_level_parameter', 
       'click_environment', 'click_deviceGroup', 'click_os', 'click_country', 'click_region', 'click_referrer_type', 'click_weekday', 
       'click_hour', 'article_crt_weekday', 'article_crt_hour',  'words_hbo', 'category_id', 'words_count', 'is_cat_hab', 'label']]

In [187]:
# # 线下验证
# del trn_user_item_feats_df_final_sample['cate_list']
# del tst_user_item_feats_df_final_sample['cate_list']
# del trn_user_item_feats_df_final_sample['cate_list_new']
# del tst_user_item_feats_df_final_sample['cate_list_new']
# del trn_user_item_feats_df_final_sample['article_id']
# del tst_user_item_feats_df_final_sample['article_id']

In [52]:
### 结果再保存！
trn_user_item_feats_df_final_sample.to_csv(save_dir + "trn_user_item_feats_df_final_sample.csv",index=False, encoding="utf8")
tst_user_item_feats_df_final_sample.to_csv(save_dir + "tst_user_item_feats_df_final_sample.csv",index=False, encoding="utf8")